# Business Understanding and Set-up

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
import os, glob
import geopandas as gpd
%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#Dashboard
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)
pd.options.display.max_seq_items = 300
#pd.options.display.max_rows = 4000
sns.set(style="white")

# Data Mining

## Data Checks

The monthly data for Berlin is composed of various files that are briefly visualized here (based on Dec 2019):

- listings.csv.gz
- listings.csv
- reviews.csv.gz
- reviews.csv
- calendar.csv.gz
- neighbourhoods.csv
- neighbourhoods.geojson

**listings.csv.gz**

In [ ]:
# Display contents of listings.csv.gz as well as its shape
data_2020_02_18_listings_gz = pd.read_csv("data/2020-02-18/listings.csv.gz")
print(data_2020_02_18_listings_gz.shape)
data_2020_02_18_listings_gz.head(3)

**listings.csv**

In [ ]:
# Display contents of listings.csv as well as its shape
data_2020_02_18_listings = pd.read_csv("data/2020-02-18/listings.csv")
print(data_2020_02_18_listings.shape)
data_2020_02_18_listings.head(2)

**reviews.csv.gz**

In [ ]:
# Display contents of reviews.csv.gz as well as its shape
data_2020_02_18_reviews_gz = pd.read_csv("data/2020-02-18/reviews.csv.gz")
print(data_2020_02_18_reviews_gz.shape)
data_2020_02_18_reviews_gz.head(2)

**reviews.csv**

In [ ]:
# Display contents of reviews.csv as well as its shape
data_2020_02_18_reviews = pd.read_csv("data/2020-02-18/reviews.csv")
print(data_2020_02_18_reviews.shape)
data_2020_02_18_reviews.head(2)

**calendar.csv.gz**

In [ ]:
# Display contents of calendar.csv.gz as well as its shape
data_2020_02_18_cal = pd.read_csv("data/2020-02-18/calendar.csv.gz")
print(data_2020_02_18_cal.shape)
data_2020_02_18_cal.head(2)

**neighbourhoods.csv**

In [ ]:
# Display contents of neighbourhoods.csv as well as its shape
data_2020_02_18_neighb = pd.read_csv("data/2020-02-18/neighbourhoods.csv")
print(data_2020_02_18_neighb.shape)
data_2020_02_18_neighb.head(2)

**neighbourhoods.geojson**

In [ ]:
# Display contents of neighbourhoods.geojson as well as its shape
data_2020_02_18_neighb_geojson = gpd.read_file('data/2020-02-18/neighbourhoods.geojson')
print(data_2020_02_18_neighb_geojson.shape)
data_2020_02_18_neighb_geojson.head(2)

## Data Import

**Create main dataset (listings on February 22nd, i.e. pre-COVID-19)**

In [ ]:
# Import dataset as DataFrame (as csv-file)
data_raw = pd.read_csv("data/2020-02-18/listings.csv.gz")

In [ ]:
# Assign data_raw to data (in order to always keep a freshly imported data_raw) and set id as index
data = data_raw.copy()
data.set_index('id', inplace=True)

# Data Cleaning

## Pre-cleaning

In [ ]:
# Display shape of "data"
data.shape

In [ ]:
# Display head(1) of "data"
data.head(1)

In [ ]:
# Display columns of "data"
#data.columns

In [ ]:
# Define columns for pre-cleaning drop
drop_columns = ['access', 'availability_30',
       'availability_60', 'availability_90',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'calendar_last_scraped',
       'calendar_updated', 'city', 
       'country', 'country_code', 
       'first_review', 
       'host_about',
       'host_name', 'host_neighbourhood', 'host_picture_url',
       'host_thumbnail_url', 'host_total_listings_count', 'host_url',
       'host_verifications',
       'interaction',
       'jurisdiction_names', 'last_review', 'last_scraped',
       'license', 'listing_url', 'market',
       'maximum_maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'medium_url', 'minimum_maximum_nights',
       'minimum_minimum_nights', 'minimum_nights_avg_ntm',
       'neighborhood_overview',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'notes',
       'picture_url', 'review_scores_rating', 'scrape_id',
       'smart_location', 'state', 'street', 
       'thumbnail_url', 'transit', 'xl_picture_url']

In [ ]:
# Drop innecessary columns
data.drop(labels=drop_columns, inplace=True, axis=1)

## Inspection

In [ ]:
# Display shape of "data"
data.shape

In [ ]:
# Display head(5) of remaining "data"
data.head(5)

In [ ]:
# Describe data (summary)
data.describe().round(2).T

In [ ]:
# List datatypes (data.info()) (pre-cleaning)
data.info()

In [ ]:
# List unique entries per column
data.nunique()

In [ ]:
# List missing values (pre-cleaning)

def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)
    
count_missing(data)

## Observations

- **host_response_rate** and **host_response_time** are unfortunately not available for half of the dataset and consequently the columns have been removed
- **review_scores** are difficult to replace if they do not exist, but at 0 they will distort the modeling. Hence, missing values are set to mean of the column
- listings without **name** and the few rows without enhanced **host information** (e.g. superhost), **bedrooms** or **bathrooms** are removed and not substantial in number
- missing values for **summary** and **description** are replaced with "" and kept in order to calculate length during feature engineering
- several features with missing values will be directly converted to 1/0 for simplification (**house_rules, security_deposit, space, cleaning_fee, monthly_price, weekly_price**)

## Data Handling

In [184]:
# Convert numeric objects to floats
data.price = [float(i.strip("$").replace(",","")) for i in data.price]

In [185]:
# Handle varying dtypes in monthly_price, weekly_price and zipcode

**Drop rows**

In [186]:
# Drop irrelevant rows with few missing values
data.dropna(subset=["name", "host_is_superhost", "bedrooms", "bathrooms", "zipcode"], inplace=True)

In [187]:
# Remove "poor" listings (price too low/too high, TBD)

**Handle missing values**

In [188]:
# Convert columns with missing values to 1/0
data.house_rules.where(data.house_rules.isnull(), 1, inplace=True)
data.house_rules.fillna(0, inplace=True)

data.security_deposit.where(data.security_deposit.isnull(), 1, inplace=True)
data.security_deposit.fillna(0, inplace=True)

data.space.where(data.space.isnull(), 1, inplace=True)
data.space.fillna(0, inplace=True)

data.cleaning_fee.where(data.cleaning_fee.isnull(), 1, inplace=True)
data.cleaning_fee.fillna(0, inplace=True)

data.monthly_price.where(data.monthly_price.isnull(), 1, inplace=True)
data.monthly_price.fillna(0, inplace=True)

data.weekly_price.where(data.weekly_price.isnull(), 1, inplace=True)
data.weekly_price.fillna(0, inplace=True)

In [189]:
# Fill missing values of "beds" with 0 and then set all with "bed_type" Real Bed to at least 1
data.beds.fillna(0, inplace=True)
data.beds = np.where((data.beds==0) & (data.bed_type=="Real Bed"), 1, data.beds)

In [190]:
# Fill review_scores with mean
data.review_scores_value.fillna(data.review_scores_value.mean(), inplace=True)
data.review_scores_checkin.fillna(data.review_scores_checkin.mean(), inplace=True)
data.review_scores_location.fillna(data.review_scores_location.mean(), inplace=True)
data.review_scores_communication.fillna(data.review_scores_communication.mean(), inplace=True)
data.review_scores_accuracy.fillna(data.review_scores_accuracy.mean(), inplace=True)
data.review_scores_cleanliness.fillna(data.review_scores_cleanliness.mean(), inplace=True)

In [191]:
# Fill missing text values with ""
data.summary.fillna("", inplace=True)
data.description.fillna("", inplace=True)

**Final check and cleaning before export**

In [183]:
# Drop further columns
data.drop(["bed_type", "host_acceptance_rate", "host_location", "host_response_rate", "host_response_time", 
           "reviews_per_month", "square_feet"], inplace=True, axis=1)

In [199]:
# List datatypes (data.info()) (pre-cleaning)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24527 entries, 3176 to 42361801
Data columns (total 51 columns):
accommodates                        24527 non-null int64
amenities                           24527 non-null object
availability_365                    24527 non-null int64
bathrooms                           24527 non-null float64
bed_type                            24527 non-null object
bedrooms                            24527 non-null float64
beds                                24527 non-null float64
calculated_host_listings_count      24527 non-null int64
cancellation_policy                 24527 non-null object
cleaning_fee                        24527 non-null int64
description                         24527 non-null object
experiences_offered                 24527 non-null object
extra_people                        24527 non-null object
guests_included                     24527 non-null int64
has_availability                    24527 non-null object
host_has_profile_

In [196]:
# List missing values (post-cleaning)

def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)
    
count_missing(data)

Series([], dtype: float64)


In [192]:
# Sort columns in dataset
data = data.reindex(sorted(data.columns, reverse=False), axis=1)

In [193]:
# Display cleaned dataset
data.head(3)

,accommodates,amenities,availability_365,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,cancellation_policy,cleaning_fee,description,experiences_offered,extra_people,guests_included,has_availability,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_since,house_rules,instant_bookable,is_business_travel_ready,is_location_exact,latitude,longitude,maximum_nights,minimum_nights,monthly_price,name,neighbourhood,number_of_reviews,number_of_reviews_ltm,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,room_type,security_deposit,space,summary,weekly_price,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3176,4,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",221,1.0,Real Bed,1.0,2.0,1,strict_14_with_grace_period,1,This beautiful first floor apartment is situa...,none,$20.00,2,t,t,3718,t,f,1.0,2008-10-19,1,f,f,t,52.53500,13.41758,1125,62,1,Fabulous Flat in great Location,Prenzlauer Berg,145,1,90.0,Apartment,f,f,t,9.0,9.0,9.0,9.0,10.0,9.0,Entire home/apt,1,1,This beautiful first floor apartment is situa...,1,10405
3309,1,"{Internet,Wifi,""Pets live on this property"",Ca...",293,1.0,Pull-out Sofa,1.0,1.0,1,strict_14_with_grace_period,1,First of all: I prefer short-notice bookings. ...,none,$18.00,1,t,t,4108,f,f,1.0,2008-11-07,1,f,f,t,52.49885,13.34906,35,7,1,BerlinSpot Schöneberg near KaDeWe,Schöneberg,27,1,28.0,Apartment,f,f,t,9.0,9.0,9.0,10.0,9.0,9.0,Private room,1,1,First of all: I prefer short-notice bookings. ...,1,10777
6883,2,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",8,1.0,Real Bed,1.0,1.0,1,moderate,1,Stay in a stylish loft on the second floor and...,none,$0.00,1,t,t,16149,t,f,1.0,2009-05-07,1,f,f,t,52.51171,13.45477,90,3,1,Stylish East Side Loft in Center with AC & 2 b...,Friedrichshain,132,8,125.0,Loft,t,f,t,10.0,10.0,10.0,10.0,10.0,10.0,Entire home/apt,1,1,,1,10243


**Export data_clean**

In [197]:
# Export dataset for further use in 2_Airbnb_EDA and 3_Airbnb_Feature_Engineering
data.to_csv(r'data_clean.csv', index = True)

**BACKUP**

In [198]:
# Import Airbnb listing data for the time period 04/2018-03/2020 (2 years)
#all_files = glob.glob(os.path.join("data", "*.csv.gz"))
#all_df = []
#for f in all_files:
#    df = pd.read_csv(f, sep=',')
#    df['file'] = f.split('/')[-1]
#    all_df.append(df)
#data_raw = pd.concat(all_df, ignore_index=True, sort=True)

In [202]:
data.number_of_reviews.value_counts()

0      4285
1      2889
2      2033
3      1532
4      1229
5       962
6       802
7       686
8       584
9       534
10      498
12      402
11      395
13      316
14      288
15      280
16      251
18      232
17      222
19      206
20      194
21      166
22      154
23      153
26      143
25      140
24      134
27      120
29      110
28      107
32      105
33      104
31      100
34       92
37       92
30       90
35       86
40       83
39       81
36       80
41       75
38       73
43       70
45       63
44       62
47       62
55       59
48       57
46       55
49       54
       ... 
522       1
285       1
299       1
315       1
331       1
347       1
395       1
459       1
491       1
507       1
571       1
237       1
284       1
316       1
348       1
428       1
520       1
456       1
530       1
440       1
259       1
275       1
355       1
276       1
356       1
388       1
404       1
468       1
548       1
830       1
526       1
334       1
229 